In [1]:
%run common.py

Planning start time: Aug 12, 07:31 (1470976300)


In [2]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'

In [3]:
d = datetime.datetime.fromtimestamp(current_time - 18 * 3600)
day_start = int(datetime.datetime.timestamp(datetime.datetime(d.year, d.month, d.day, 18, 0, 0))) + 24 * 3600
print(time.ctime(current_time))
print(time.ctime(day_start))

Fri Aug 12 07:31:40 2016
Fri Aug 12 18:00:00 2016


In [4]:
team_plan['dt_start'] = team_plan.time_start.apply(datetime.datetime.fromtimestamp)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
team_start_stations = team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team')
team_plan['start_st'] = team_plan.team.map(team_start_stations.st_from_name)
team_plan.depot.fillna(team_plan.start_st, inplace=True)

In [5]:
ct = datetime.datetime.fromtimestamp(current_time)
dt = datetime.datetime.fromtimestamp(day_start)

In [6]:
team_plan['hour_start'] = team_plan.time_start.apply(lambda x: 3600 * int(x / 3600))
team_plan['hour_start_norm'] = team_plan.hour_start.apply(nice_time)

In [13]:
# Total team depart
team_cols = ['team', 'st_from_name', 'st_to_name', 'time_start_norm', 'hour_start_norm', 'state', 'loco', 'depot_name']
a = team_plan[(team_plan.st_from_name == st_name) 
              & (team_plan.state.isin([0, 1]))
              & (team_plan.time_start >= day_start)
              & (team_plan.time_start < day_start + 24 * 3600)].sort_values('time_start')
print(a.groupby(['st_to_name', 'state']).team.count())
b = a.groupby(['st_to_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c = b.pivot('st_to_name', 'hour_start_norm', 'team').fillna(0)
c.transpose()

st_to_name       state
БАТАРЕЙНАЯ       0         7
                 1        56
ВОЕННЫЙ ГОРОДОК  0         2
                 1         2
ГОНЧАРОВО        1        46
Name: team, dtype: int64


st_to_name,БАТАРЕЙНАЯ,ВОЕННЫЙ ГОРОДОК,ГОНЧАРОВО
hour_start_norm,,,
"Aug 12, 18:00",0,0,1
"Aug 12, 19:00",0,0,1
"Aug 12, 20:00",2,0,1
"Aug 12, 21:00",4,0,2
"Aug 12, 22:00",1,0,4
"Aug 12, 23:00",3,1,5
"Aug 13, 00:00",5,0,1
"Aug 13, 01:00",6,1,2
"Aug 13, 02:00",4,1,2


In [12]:
b_dep = a.groupby(['depot_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c_dep = b_dep.pivot('depot_name', 'hour_start_norm', 'team').fillna(0)
print(c_dep.sum(axis = 1).sort_values(ascending=False))
c_dep.transpose()

depot_name
ЗИМА                     45
ИРКУТСК-СОРТИРОВОЧНЫЙ    44
УЛАН-УДЭ                  5
СЛЮДЯНКА I                3
НИЖНЕУДИНСК               2
dtype: float64


depot_name,ЗИМА,ИРКУТСК-СОРТИРОВОЧНЫЙ,НИЖНЕУДИНСК,СЛЮДЯНКА I,УЛАН-УДЭ
hour_start_norm,,,,,
"Aug 12, 18:00",0,0,0,0,1
"Aug 12, 19:00",0,1,0,0,0
"Aug 12, 20:00",3,0,0,0,0
"Aug 12, 21:00",4,1,0,0,1
"Aug 12, 22:00",1,3,0,0,1
"Aug 12, 23:00",3,5,0,1,0
"Aug 13, 00:00",5,1,0,0,0
"Aug 13, 01:00",2,4,2,1,0
"Aug 13, 02:00",1,5,0,1,0


In [9]:
st1 = 'БАТАРЕЙНАЯ'
d = team_plan[(team_plan.st_from_name == st_name) & (team_plan.st_to_name == st1)
           & (team_plan.state.isin([0, 1]))
           & (team_plan.time_start >= day_start)
           & (team_plan.time_start < day_start + 24 * 3600)].sort_values('time_start')
d[d.state == 1].hour_start_norm.value_counts().sort_index()
#d[d.state == 1][team_cols].head(10)

Aug 12, 20:00    2
Aug 12, 21:00    4
Aug 12, 22:00    1
Aug 12, 23:00    3
Aug 13, 00:00    5
Aug 13, 01:00    4
Aug 13, 02:00    4
Aug 13, 03:00    3
Aug 13, 04:00    4
Aug 13, 05:00    3
Aug 13, 06:00    2
Aug 13, 07:00    5
Aug 13, 08:00    1
Aug 13, 09:00    4
Aug 13, 10:00    4
Aug 13, 11:00    1
Aug 13, 14:00    1
Aug 13, 15:00    1
Aug 13, 16:00    1
Aug 13, 17:00    3
Name: hour_start_norm, dtype: int64

In [10]:
team_plan['type'] = team_plan.team.apply(lambda x: int(str(x)[:1]))
print(time.ctime(current_time))
team_plan[(team_plan['type'] == 7) & (team_plan.state.isin([0, 1]) & (team_plan.time_start <= current_time + 30 * 3600))]\
        .drop_duplicates('team').hour_start_norm.value_counts().sort_index()

Fri Aug 12 07:31:40 2016


Aug 12, 08:00     6
Aug 12, 09:00    42
Aug 12, 10:00    27
Aug 12, 11:00    36
Aug 12, 12:00    27
Aug 12, 13:00    36
Aug 12, 14:00    34
Aug 12, 15:00    25
Aug 12, 16:00    23
Aug 12, 17:00    17
Aug 12, 18:00    17
Aug 12, 19:00    19
Aug 12, 20:00    16
Aug 12, 21:00    17
Aug 12, 22:00    19
Aug 12, 23:00    15
Aug 13, 00:00    26
Aug 13, 01:00    24
Aug 13, 02:00    30
Aug 13, 03:00    46
Aug 13, 04:00    24
Aug 13, 05:00    39
Aug 13, 06:00    31
Aug 13, 07:00    44
Aug 13, 08:00    47
Aug 13, 09:00    27
Aug 13, 10:00    11
Aug 13, 11:00    14
Aug 13, 12:00    11
Name: hour_start_norm, dtype: int64

In [11]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
irk_t = team_plan[(team_plan.st_from_name == st_name) & (team_plan.state.isin([0, 1]))
         & (team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)]
irk = team_plan[(team_plan.team.isin(irk_t.team)) & (team_plan.state.isin([0, 1]))].copy(deep=True)
mins = irk.groupby('team').time_start.min()
irk['end_trip'] = (irk.team != irk.team.shift(-1)) | (irk.state != irk.state.shift(-1)) | (irk.loco != irk.loco.shift(-1))
irk['start_trip'] = (irk.team != irk.team.shift(1)) | (irk.state != irk.state.shift(1)) | (irk.loco != irk.loco.shift(1))
irk[['team', 'start_trip', 'end_trip']]
irk[(irk.start_trip == True) | (irk.end_trip == True)][['team', 'st_from_name', 'st_to_name', 'state', 'loco']]
starts = irk[(irk.start_trip == True)][['team', 'st_from_name', 'time_start', 'time_start_norm', 'state', 'loco']].reset_index()
ends = irk[(irk.end_trip == True)][['st_to_name', 'time_end', 'time_end_norm']].reset_index()
cols = ['team', 'st_from_name', 'st_to_name', 'time_start', 'time_end', 'time_start_norm', 'time_end_norm', 'state', 'loco']
trips = pd.concat([starts, ends], axis=1)[cols]
trips[(trips.st_from_name == st_name)].st_to_name.value_counts()

ЗИМА                     33
СЛЮДЯНКА I               29
УЛАН-УДЭ                 13
МЕГЕТ                     8
БАТАРЕЙНАЯ                6
СУХОВСКАЯ                 5
КАЯ                       5
ИРКУТСК-СОРТИРОВОЧНЫЙ     4
ЗУЙ                       2
КИТОЙ                     2
НИЖНЕУДИНСК               2
СУХОВСКАЯ-ЮЖНАЯ           1
КАСЬЯНОВКА                1
ЧЕРЕМХОВО                 1
ГОНЧАРОВО                 1
Name: st_to_name, dtype: int64